In [13]:
import requests
import time

In [9]:
print(requests.get("http://localhost:8010/RestPLCService/getint?address=4020").json()["Value"])

1510


In [10]:
requests.get("http://localhost:8010/RestPLCService/setint?address=4010&value=%d" % 1209)

<Response [200]>

In [12]:
requests.get("http://localhost:8010/RestPLCService/setint?address=4010&value=%d" % 1510)

<Response [200]>

In [31]:
li_th_pos = [1340, 1055, 740, 435, 1510]
if requests.get("http://localhost:8010/RestPLCService/getint?address=4020").json()["Value"] != 1510:
    requests.get("http://localhost:8010/RestPLCService/setint?address=4010&value=%d" % 1510)
    time.sleep(10)
    print("The gantry's position is:", requests.get("http://localhost:8010/RestPLCService/getint?address=4020").json()["Value"])
    # call rgbd and thermal capture
for pos in li_th_pos:
    requests.get("http://localhost:8010/RestPLCService/setint?address=4010&value=%d" % pos)
    time.sleep(10)
    print("The gantry's position is:",requests.get("http://localhost:8010/RestPLCService/getint?address=4020").json()["Value"])
    # call rgbd and thermal capture

The gantry's position is: 1510
The gantry's position is: 1340
The gantry's position is: 1055
The gantry's position is: 740
The gantry's position is: 435
The gantry's position is: 1427
